# Name: Margaret Nguyen

# Data Aggregation: Virginia Crash Data and ACS Data

**Assignment: Retrieve crash data from 2017 to 2021 for Virginia. Try to find a way to organize the data by municipality so that we can later merge it with county subdivision data from the ACS 5-year estimates. You can access the Virginia crash data details [here](https://www.virginiaroads.org/datasets/VDOT::crash-data-1/explore?layer=1) and the Virginia crash data basic [here](https://www.virginiaroads.org/datasets/VDOT::crash-data-1/explore?layer=0&location=37.968178%2C-79.499811%2C7.93).**

**Here is the link to the Virginia Crash Data Dictionary: [link](https://vdot.maps.arcgis.com/sharing/rest/content/items/490d94134bf342c281ded44ca22a8630/data).**

**Here is the link to the Virginia Jurisdictions: [link](https://vdot.maps.arcgis.com/home/item.html?id=d28169c81fd94122a858f47e6607d29b#overview).**

## Credit:

The following code is based on the work of my supervisor, Mitch Shiles. The original code can be found at this link: [Mitch Shiles' GitHub](https://github.com/rmshiles/Carlisle-Local-Crash-Analysis/blob/main/1.%20Municupal%20Crash%20Data%20Aggregation%20.ipynb).

In [1]:
# Import necessary libraries 
import requests
import json, csv
import timeit
from pathlib import Path  

# Import data handling libraries 
import numpy as np
import pandas as pd
import requests
import json

# Import graphing libraries 
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from pathlib import Path  

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:,.6f}'.format

## Set Global Variables

In [2]:
# Set Global Variables

# Set the time period in which to look at 
START_YEAR = 2017
END_YEAR = 2021

## Query ACS Data for Municipalities 

In [3]:
# Query ACS data for Municipalities 

# Census API
HOST = 'https://api.census.gov/data'

# Year to get data for 
YEAR = '{}'.format(END_YEAR)# 2005,2010,2015,2020  

# Survey to Access data from (ACS 5 year estimates)
DATA_SET = 'acs/acs5'
BASE_URL = '/'.join([HOST, YEAR, DATA_SET])

# Create an empty dictionary for predicates 
predicates = {}

# VARIABLES
# Population Estimate:  B01003_001E
# Population Margin of error: B01003_001M
# Bike to work Estimate: B08006_014E
# Bike to work Margin of error: B08006_014M
# Walk to work Estimate:B08006_015E
# Walk to work Margin of error:B08006_015M
# Drive to work alone Estimate: B08006_003E
# Drive to work alone Margin of error: B08006_003M
# Carpool to work Estimate:B08006_004E
# Carpool to work Margin: 
# Public transit to work Estimate: B08006_008E
# Public transit to work Margine of Error: B08006_008E
# Other to Work 
# Poverty Estimate:
# Poverty Margin of error:

get_vars = ['NAME',
            'B01001_001E',
            'B08006_014E',
            'B08006_014M',
            'B08006_015E',
            'B08006_015M',
            'B08006_003E',
            'B08006_003M',
            'B08006_004E',
            'B08006_004M',
            'B08006_008E',
            'B08006_008M']

predicates['get']=','.join(get_vars)

# Set sub geographies to get data for ('county','Place','county subdivision') * means get all 
predicates['for']='county subdivision:*'

# Set geography that contains sub geographies  (51 = Virginia)
predicates['in']='state:51'

# Assemble the API query 
r = requests.get(BASE_URL, params = predicates)

# Print the query URL 
print(BASE_URL,predicates)

https://api.census.gov/data/2021/acs/acs5 {'get': 'NAME,B01001_001E,B08006_014E,B08006_014M,B08006_015E,B08006_015M,B08006_003E,B08006_003M,B08006_004E,B08006_004M,B08006_008E,B08006_008M', 'for': 'county subdivision:*', 'in': 'state:51'}


In [4]:
# Print the length and first thousand characters to see what you got 
print(len(r.text))
print(r.text[0:1000])

71024
[["NAME","B01001_001E","B08006_014E","B08006_014M","B08006_015E","B08006_015M","B08006_003E","B08006_003M","B08006_004E","B08006_004M","B08006_008E","B08006_008M","state","county","county subdivision"],
["District 1, Accomack County, Virginia","3309","81","107","49","45","932","185","123","96","0","13","51","001","91196"],
["District 2, Accomack County, Virginia","3974","0","13","203","158","1098","222","55","61","0","13","51","001","91346"],
["District 3, Accomack County, Virginia","3392","0","13","79","93","1006","250","342","219","0","13","51","001","91496"],
["District 4, Accomack County, Virginia","3330","28","40","7","9","1172","254","95","51","7","10","51","001","91646"],
["District 5, Accomack County, Virginia","4764","0","13","0","13","1503","350","727","345","78","78","51","001","91796"],
["District 6, Accomack County, Virginia","3251","26","23","105","86","908","309","99","103","6","9","51","001","91946"],
["District 7, Accomack County, Virginia","3465","0","13","10","

### Caution: 5 unique values in NAME column before splitting:
- District 1, West Luray, Page County
- District 2, East Luray, Page County
- District 3, Marksville, Page County
- District 4, Newport, Page County
- District 5, Shenandoah, Page County

In [5]:
# Place the Queried ACS data into a data frame 

# Set the column names to the first row of data from the query 
column_names=r.json()[0:1][0]

# Set the data to everything after the first row and convert to an array to flatten it
ACS_DATA= r.json()[1:]
ACS_data = np.array(ACS_DATA)

# Create the pandas data frame 
ACS_MUNI_DF = pd.DataFrame(columns=column_names , data = ACS_data)

# Reset the index of the data frame
ACS_MUNI_DF.reset_index()

# Rename the columns 
ACS_MUNI_DF.rename(columns ={"B01001_001E":"POPULATION",
                    "B08006_014E":'BIKE_TO_WORK_EST',
                    "B08006_014M":"BIKE_TO_WORK_MARG",
                    "B08006_015E":"WALK_TO_WORK_EST",
                    "B08006_015M":"WALK_TO_WORK_MARG",
                    'B08006_003E':"DRIVE_SOLO_TO_WORK_EST",
                    'B08006_003M':"DRIVE_SOLO_TO_WORK_MARG",
                    'B08006_004E':"CARPOOL_TO_WORK_EST",
                    'B08006_004M':"CARPOOL_TO_WORK_MARG",
                    'B08006_008E':"PUBTRANS_TO_WORK_EST",
                    'B08006_008M':"PUBTRANS_TO_WORK_MARG",
                    "county subdivision":"county_subdivision"}, inplace=True)

# Convert the NAME column to strings 
ACS_MUNI_DF['NAME'] = ACS_MUNI_DF["NAME"].astype(str)

# Remove Virginia from NAME This is over redundant since all data will be from Virginia
ACS_MUNI_DF['NAME'] = ACS_MUNI_DF.NAME.replace({', Virginia':''}, regex=True)

# Create separate Name variables for county and municipality 
ACS_MUNI_DF[['MUNI_NAME','COUNTY_NAME']]=ACS_MUNI_DF['NAME'].str.split(',', n=1, expand=True)

# Clean value of Virginia Beach city Beach city
ACS_MUNI_DF.loc[ACS_MUNI_DF['NAME'] == 'Virginia Beach city Beach city', 'MUNI_NAME'] = 'Virginia Beach city'
ACS_MUNI_DF.loc[ACS_MUNI_DF['NAME'] == 'Virginia Beach city Beach city', 'COUNTY_NAME'] = 'Virginia Beach city'
    
# Convert the variables to integers
ACS_MUNI_DF[["POPULATION",
                "BIKE_TO_WORK_EST",
                "BIKE_TO_WORK_MARG",
                "WALK_TO_WORK_EST",
                "WALK_TO_WORK_MARG",
                "DRIVE_SOLO_TO_WORK_EST",
                "DRIVE_SOLO_TO_WORK_MARG",
                "CARPOOL_TO_WORK_EST",
                "CARPOOL_TO_WORK_MARG",
                "PUBTRANS_TO_WORK_EST",
                "PUBTRANS_TO_WORK_MARG",
                "state",                 
                "county",                
                "county_subdivision"]] = ACS_MUNI_DF[["POPULATION",
                                            "BIKE_TO_WORK_EST",
                                            "BIKE_TO_WORK_MARG",
                                            "WALK_TO_WORK_EST",
                                            "WALK_TO_WORK_MARG",
                                            "DRIVE_SOLO_TO_WORK_EST",
                                            "DRIVE_SOLO_TO_WORK_MARG",
                                            "CARPOOL_TO_WORK_EST",
                                            "CARPOOL_TO_WORK_MARG",
                                            "PUBTRANS_TO_WORK_EST",
                                            "PUBTRANS_TO_WORK_MARG",
                                            "state",                 
                                            "county",                
                                            "county_subdivision"]].astype(int)

# Show the length of an object referred to the dataframe
print(len(ACS_MUNI_DF))

# Show the dataframe
ACS_MUNI_DF.head()

552


,NAME,POPULATION,BIKE_TO_WORK_EST,BIKE_TO_WORK_MARG,WALK_TO_WORK_EST,WALK_TO_WORK_MARG,DRIVE_SOLO_TO_WORK_EST,DRIVE_SOLO_TO_WORK_MARG,CARPOOL_TO_WORK_EST,CARPOOL_TO_WORK_MARG,PUBTRANS_TO_WORK_EST,PUBTRANS_TO_WORK_MARG,state,county,county_subdivision,MUNI_NAME,COUNTY_NAME
0,"District 1, Accomack County",3309,81,107,49,45,932,185,123,96,0,13,51,1,91196,District 1,Accomack County
1,"District 2, Accomack County",3974,0,13,203,158,1098,222,55,61,0,13,51,1,91346,District 2,Accomack County
2,"District 3, Accomack County",3392,0,13,79,93,1006,250,342,219,0,13,51,1,91496,District 3,Accomack County
3,"District 4, Accomack County",3330,28,40,7,9,1172,254,95,51,7,10,51,1,91646,District 4,Accomack County
4,"District 5, Accomack County",4764,0,13,0,13,1503,350,727,345,78,78,51,1,91796,District 5,Accomack County


In [6]:
# Make a copy of ACS_MUNI_DF
virginia_acs = ACS_MUNI_DF.copy()

## Importing Virginia Crash Data from 2017 to 2021

**The following is my code:**

In [9]:
# Load dataframes
df_vdot_detailed = pd.read_csv('./data/virginia_crash_data_detailed.csv', low_memory=False)

df_vdot_basic = pd.read_csv('./data/virginia_crash_data_basic.csv')

## Exploratory Data Analysis (EDA)

### Virginia Crash Data Detailed

In [10]:
# View dataframe
df_vdot_detailed.head()

,OBJECTID,DOCUMENT_NBR,CRASH_YEAR,CRASH_SEVERITY,DRIVER_VEHICLENUMBER,DRIVER_INJURYTYPE,DRIVER_AGE,DRIVER_GENDER,DRIVER_ACTION_TYPE_CD,DRIVER_AIRBAG_DEPLOYMENT,DRIVER_ALCOHOL_TEST_TYPE_CD,DRIVER_CONDITION_TYPE_CD,DRIVER_DISTRACTION_TYPE_CD,DRIVER_DRINKING_TYPE_CD,DRIVER_DRUG_USE_CD,DRIVER_EJECTED_FROM_VEHICLE,DRIVER_EMS_TRANSPORT_IND,DRIVER_FLED_SCENE_IND,DRIVER_SAFETY_EQUIP_USED,DRIVER_VIS_OBSCURED_TYPE_CD,SUMMONS_ISSUED_CD,VEHICLENUMBER,VEHICLE_BODY_TYPE_CD,VEHICLE_MAKE_NM,VEHICLE_MANEUVER_TYPE_CD,VEHICLE_MODEL_NM,VEHICLE_YEAR_NBR,COMM_CARGO_BODY_TYPE_CD,COMM_VEHICLE_BODY_TYPE_CD,SPEED_BEFORE,SPEED_POSTED,SPEED_MAX_SAFE,FIRST_CRASH_EVENT_CD,SECOND_CRASH_EVENT_CD,THIRD_CRASH_EVENT_CD,FOURTH_CRASH_EVENT_CD,MOST_HARMFUL_CRASH_EVENT_CD,INITIAL_VEH_IMPACT_AREA_CD,DIRECTION_OF_TRAVEL_CD,PASS_VEHICLENUMBER,PASS_INJURYTYPE,PASS_AGE,PASS_GENDER,PASS_SAFETY_EQUIP_USED,PASS_AIRBAG_DEPLOYMENT,PASS_EJECTED_FROM_VEHICLE,PASS_EMS_TRANSPORT_IND,BIKE_VEHICLENUMBER,BIKE_INJURYTYPE,BIKE_AGE,BIKE_GENDER,PED_NUMBER,PED_INJURYTYPE,PED_AGE,PED_GENDER,PED_ACTION,PED_DRINK,PED_COND,PED_AL_TEST,PED_DRUG,PED_RFLCT,NODE_TOTAADT,SEG_TOTAADT,BASETYPEDESC,CURBGUTTERDESC,RURALURBANDESC,ACCESS_CONTROL,SURFACEDESC,GOVCONDESC,SIDEWALKDESC,TRUCKCOMMR,NUMBEROFLANE,CARSPEEDLIMIT,PAVEMENT_CONDITIONVALUE,PAVEMENT_ROUGHNESSVALUE,PAVEMENT_WIDTH,RIGHTSHOULDERWIDTH,LEFTSHOULDERWIDTH,MEDIAN_TYPE,MEDIAN_TYPEDESC,MEDIAN_LEFTSHOULDERWIDTH,MEDIAN_RIGHTSHOULDERWIDTH,MEDIAN_WIDTHMAX,MEDIAN_WIDTHMIN,LOCAL_CASE_CD,ROUTE_OR_STREET_NM
0,1,153435019,2015,O,1;2;3,PDO;PDO;PDO,26;30;47,Male;Male;Male,24. Fail to Stop at Through Highway - No Sign;...,1. Deployed - Front;2. Not Deployed;2. Not Dep...,4. No Test;4. No Test;4. No Test,1. No Defects;1. No Defects;1. No Defects,11. Other; n/a; n/a,1. No Drinking;1. No Drinking;1. No Drinking,2. No;2. No;2. No,1. Not Ejected;1. Not Ejected;1. Not Ejected,No;No;No,No;No;No,3. Lap and Shoulder Belt;3. Lap and Shoulder B...,13. Other;1. Not Obscured;1. Not Obscured,1. Yes;2. No;2. No,1;2;3,1. Passenger car;1. Passenger car; n/a,VW;MERCEDES-BENZ;HYUNDAI,1. Going Straight Ahead;8. Stopped in Traffic ...,JETTA;C CLASS;ELANTRA,1998;2010;2008,n/a; n/a; n/a,n/a; n/a; n/a,35;0;0,35;35;35,35;35;35,20. Motor Vehicle In Transport;20. Motor Vehic...,n/a; n/a; n/a,n/a; n/a; n/a,n/a; n/a; n/a,20. Motor Vehicle In Transport;20. Motor Vehic...,12;6;6,South;South;South,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"9,275.000000","10,559.000000",1.000000,NaN,2.000000,1.000000,NaN,4,NaN,1.000000,3.000000,NaN,0.000000,189.000000,44.000000,4.000000,4.000000,2.000000,NaN,0.000000,0.000000,0.000000,0.000000,15120312,352 REMOUNT RD
1,2,153405153,2015,O,1;2,PDO;PDO,32;20,Male;Female,12. Following Too Close;1. No Improper Action,2. Not Deployed;2. Not Deployed,n/a; n/a,1. No Defects;1. No Defects,14. No Distraction;14. No Distraction,1. No Drinking;1. No Drinking,2. No;2. No,1. Not Ejected;1. Not Ejected,No;No,No;No,3. Lap and Shoulder Belt;3. Lap and Shoulder Belt,1. Not Obscured;1. Not Obscured,1. Yes;2. No,1;2,3. Van;1. Passenger car,HONDA;HYUNDAI,1. Going Straight Ahead;8. Stopped in Traffic ...,ODYSSEY;SONATA,2010;2015,n/a; n/a,n/a; n/a,15;0,55;55,0;0,20. Motor Vehicle In Transport;20. Motor Vehic...,n/a; n/a,n/a; n/a,n/a; n/a,20. Motor Vehicle In Transport;20. Motor Vehic...,12;6,North;North,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"76,966.000000",8.000000,2.000000,4.000000,3.000000,NaN,1,1.000000,5.000000,3.000000,55.000000,0.000000,84.000000,0.000000,0.000000,0.000000,1.000000,8.000000,0.000000,0.000000,4.000000,4.000000,DIV715174404,I-395 NB
2,3,153055246,2015,B,1;2;3;4,B;PDO;PDO;PDO,20;61;68;43,Female;Male;Male;Male,11. Did Not Have Right-of-Way;1. No Improper A...,1. Deployed - Front;2. Not Deployed;2. Not Dep...,n/a; n/a; n/a; n/a,1. No Defects;1. No Defects;1. No Defects;1. N...,14. No Distraction;14. No Distraction;14. No D...,1. No Drinking;1. No Drinking;1. No

In [11]:
# View dataframe
df_vdot_detailed.tail()

,OBJECTID,DOCUMENT_NBR,CRASH_YEAR,CRASH_SEVERITY,DRIVER_VEHICLENUMBER,DRIVER_INJURYTYPE,DRIVER_AGE,DRIVER_GENDER,DRIVER_ACTION_TYPE_CD,DRIVER_AIRBAG_DEPLOYMENT,DRIVER_ALCOHOL_TEST_TYPE_CD,DRIVER_CONDITION_TYPE_CD,DRIVER_DISTRACTION_TYPE_CD,DRIVER_DRINKING_TYPE_CD,DRIVER_DRUG_USE_CD,DRIVER_EJECTED_FROM_VEHICLE,DRIVER_EMS_TRANSPORT_IND,DRIVER_FLED_SCENE_IND,DRIVER_SAFETY_EQUIP_USED,DRIVER_VIS_OBSCURED_TYPE_CD,SUMMONS_ISSUED_CD,VEHICLENUMBER,VEHICLE_BODY_TYPE_CD,VEHICLE_MAKE_NM,VEHICLE_MANEUVER_TYPE_CD,VEHICLE_MODEL_NM,VEHICLE_YEAR_NBR,COMM_CARGO_BODY_TYPE_CD,COMM_VEHICLE_BODY_TYPE_CD,SPEED_BEFORE,SPEED_POSTED,SPEED_MAX_SAFE,FIRST_CRASH_EVENT_CD,SECOND_CRASH_EVENT_CD,THIRD_CRASH_EVENT_CD,FOURTH_CRASH_EVENT_CD,MOST_HARMFUL_CRASH_EVENT_CD,INITIAL_VEH_IMPACT_AREA_CD,DIRECTION_OF_TRAVEL_CD,PASS_VEHICLENUMBER,PASS_INJURYTYPE,PASS_AGE,PASS_GENDER,PASS_SAFETY_EQUIP_USED,PASS_AIRBAG_DEPLOYMENT,PASS_EJECTED_FROM_VEHICLE,PASS_EMS_TRANSPORT_IND,BIKE_VEHICLENUMBER,BIKE_INJURYTYPE,BIKE_AGE,BIKE_GENDER,PED_NUMBER,PED_INJURYTYPE,PED_AGE,PED_GENDER,PED_ACTION,PED_DRINK,PED_COND,PED_AL_TEST,PED_DRUG,PED_RFLCT,NODE_TOTAADT,SEG_TOTAADT,BASETYPEDESC,CURBGUTTERDESC,RURALURBANDESC,ACCESS_CONTROL,SURFACEDESC,GOVCONDESC,SIDEWALKDESC,TRUCKCOMMR,NUMBEROFLANE,CARSPEEDLIMIT,PAVEMENT_CONDITIONVALUE,PAVEMENT_ROUGHNESSVALUE,PAVEMENT_WIDTH,RIGHTSHOULDERWIDTH,LEFTSHOULDERWIDTH,MEDIAN_TYPE,MEDIAN_TYPEDESC,MEDIAN_LEFTSHOULDERWIDTH,MEDIAN_RIGHTSHOULDERWIDTH,MEDIAN_WIDTHMAX,MEDIAN_WIDTHMIN,LOCAL_CASE_CD,ROUTE_OR_STREET_NM
1079554,1079555,232225317,2023,O,1;2,PDO;PDO,17;29,Female;Female,42. Improper or Unsafe Lane Change;1. No Impro...,2. Not Deployed;2. Not Deployed,4. No Test;4. No Test,1. No Defects;1. No Defects,14. No Distraction;14. No Distraction,1. No Drinking;1. No Drinking,2. No;2. No,1. Not Ejected;1. Not Ejected,No;No,No;No,3. Lap and Shoulder Belt;3. Lap and Shoulder Belt,n/a; n/a,2. No;2. No,1;2,22. Truck (SUV);22. Truck (SUV),NISSAN;TOYOTA,14. Changing Lanes;1. Going Straight Ahead,MURANO;RAV4,2017;2021,n/a; n/a,n/a; n/a,45;45,55;55,n/a;n/a,20. Motor Vehicle In Transport;20. Motor Vehic...,n/a; n/a,n/a; n/a,n/a; n/a,20. Motor Vehicle In Transport;20. Motor Vehic...,8;1,East;East,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SO230014851,43375 OLD OX RD
1079555,1079556,232545121,2023,B,1;2,PDO;PDO,56;35,Female;Female,37. Other;1. No Improper Action,2. Not Deployed;2. Not Deployed,n/a; n/a,1. No Defects;1. No Defects,14. No Distraction;14. No Distraction,1. No Drinking;1. No Drinking,2. No;2. No,1. Not Ejected;1. Not Ejected,No;No,No;No,3. Lap and Shoulder Belt;3. Lap and Shoulder Belt,1. Not Obscured;1. Not Obscured,2. No;2. No,1;2,13. Bus - School Bus;1. Passenger car,CHEV;SUBA,1. Going Straight Ahead;11. Parked,CARGO CUTAWY;CROSSTREK,2013;2021,n/a; n/a,n/a; n/a,10;0,10;10,0;0,20. Motor Vehicle In Transport;6. Parked Vehicle,6. Parked Vehicle;20. Motor Vehicle In Transport,n/a; n/a,n/a; n/a,20. Motor Vehicle In Transport;20. Motor Vehic...,12;6,South;South,1,B,8,Female,9. n/a,2. Not Deployed,1. Not Ejected,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23052889,7311 RICHMOND RD
1079556,1079557,232605076,2023,B,1,B,27,Male,3. Exceeded Safe Speed But Not Speed Limit,2. Not Deployed,n/a,1. No Defects,14. No Distraction,1. No Drinking,2. No,1. Not Ejected,No,No,3. Lap and Shoulder Belt,"2. Rain, Snow, etc. on Windshield",1. Yes,1,2. Pick-up Truck,FORD,9. Ran Off Road - Right,F-350,2022,n/a,n/a,45,55,35,28. Ran Off Road,2. Trees,n/a,n/a,2. Trees,12,West,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DIV423080238,ROUTE 80
1079557,1079558,231635040,2023,O,1,PDO,18,Female,1. No Improper Action,2. 

In [12]:
df_vdot_detailed.shape, df_vdot_detailed.columns, df_vdot_detailed.dtypes

((1079559, 86),
 Index(['OBJECTID', 'DOCUMENT_NBR', 'CRASH_YEAR', 'CRASH_SEVERITY', 'DRIVER_VEHICLENUMBER', 'DRIVER_INJURYTYPE', 'DRIVER_AGE', 'DRIVER_GENDER', 'DRIVER_ACTION_TYPE_CD', 'DRIVER_AIRBAG_DEPLOYMENT', 'DRIVER_ALCOHOL_TEST_TYPE_CD', 'DRIVER_CONDITION_TYPE_CD', 'DRIVER_DISTRACTION_TYPE_CD', 'DRIVER_DRINKING_TYPE_CD', 'DRIVER_DRUG_USE_CD', 'DRIVER_EJECTED_FROM_VEHICLE', 'DRIVER_EMS_TRANSPORT_IND', 'DRIVER_FLED_SCENE_IND', 'DRIVER_SAFETY_EQUIP_USED', 'DRIVER_VIS_OBSCURED_TYPE_CD', 'SUMMONS_ISSUED_CD', 'VEHICLENUMBER', 'VEHICLE_BODY_TYPE_CD', 'VEHICLE_MAKE_NM', 'VEHICLE_MANEUVER_TYPE_CD', 'VEHICLE_MODEL_NM', 'VEHICLE_YEAR_NBR', 'COMM_CARGO_BODY_TYPE_CD', 'COMM_VEHICLE_BODY_TYPE_CD', 'SPEED_BEFORE', 'SPEED_POSTED', 'SPEED_MAX_SAFE', 'FIRST_CRASH_EVENT_CD', 'SECOND_CRASH_EVENT_CD', 'THIRD_CRASH_EVENT_CD', 'FOURTH_CRASH_EVENT_CD', 'MOST_HARMFUL_CRASH_EVENT_CD', 'INITIAL_VEH_IMPACT_AREA_CD', 'DIRECTION_OF_TRAVEL_CD', 'PASS_VEHICLENUMBER', 'PASS_INJURYTYPE', 'PASS_AGE', 'PASS_GENDER'

In [13]:
# Show a summary of the number of missing values (NaN or null) in each column of the df_vdot_detailed DataFrame.
df_vdot_detailed.isna().sum()

OBJECTID                             0
DOCUMENT_NBR                         0
CRASH_YEAR                           0
CRASH_SEVERITY                       0
DRIVER_VEHICLENUMBER                 0
DRIVER_INJURYTYPE                 4257
DRIVER_AGE                       21000
DRIVER_GENDER                        0
DRIVER_ACTION_TYPE_CD                0
DRIVER_AIRBAG_DEPLOYMENT             0
DRIVER_ALCOHOL_TEST_TYPE_CD          0
DRIVER_CONDITION_TYPE_CD             0
DRIVER_DISTRACTION_TYPE_CD           0
DRIVER_DRINKING_TYPE_CD              0
DRIVER_DRUG_USE_CD                   0
DRIVER_EJECTED_FROM_VEHICLE          0
DRIVER_EMS_TRANSPORT_IND             0
DRIVER_FLED_SCENE_IND                0
DRIVER_SAFETY_EQUIP_USED             0
DRIVER_VIS_OBSCURED_TYPE_CD          0
SUMMONS_ISSUED_CD                    0
VEHICLENUMBER                        0
VEHICLE_BODY_TYPE_CD                 0
VEHICLE_MAKE_NM                   2734
VEHICLE_MANEUVER_TYPE_CD             0
VEHICLE_MODEL_NM         

### Virginia Crash Data Basic

In [14]:
# View dataframe
df_vdot_basic.head()

,X,Y,OBJECTID,DOCUMENT_NBR,LOCAL_CASE_CD,CRASH_YEAR,CRASH_DT,CRASH_MILITARY_TM,CRASH_SEVERITY,K_PEOPLE,A_PEOPLE,B_PEOPLE,C_PEOPLE,PERSONS_INJURED,PEDESTRIANS_KILLED,PEDESTRIANS_INJURED,VEH_COUNT,COLLISION_TYPE,WEATHER_CONDITION,LIGHT_CONDITION,ROADWAY_SURFACE_COND,RELATION_TO_ROADWAY,ROADWAY_ALIGNMENT,ROADWAY_SURFACE_TYPE,ROADWAY_DEFECT,ROADWAY_DESCRIPTION,INTERSECTION_TYPE,TRAFFIC_CONTROL_TYPE,TRFC_CTRL_STATUS_TYPE,WORK_ZONE_RELATED,WORK_ZONE_LOCATION,WORK_ZONE_TYPE,SCHOOL_ZONE,FIRST_HARMFUL_EVENT,FIRST_HARMFUL_EVENT_LOC,ROUTE_OR_STREET_NM,ALCOHOL_NOTALCOHOL,BELTED_UNBELTED,BIKE_NONBIKE,DISTRACTED_NOTDISTRACTED,ANIMAL,DROWSY_NOTDROWSY,DRUG_NODRUG,GR_NOGR,HITRUN_NOT_HITRUN,LGTRUCK_NONLGTRUCK,MOTOR_NONMOTOR,PED_NONPED,SPEED_DIFF_MAX,SPEED_NOTSPEED,RD_TYPE,INTERSECTION_ANALYSIS,SENIOR_NOTSENIOR,YOUNG_NOTYOUNG,MAINLINE_YN,NIGHT,VDOT_DISTRICT,JURIS_CODE,PHYSICAL_JURIS,FUN,FAC,AREA_TYPE,SYSTEM,VSP,OWNERSHIP,PLAN_DISTRICT,MPO_NAME,RTE_NM,RNS_MP,NODE,OFFSET,LAT,LON
0,-78.870556,38.645544,1,152915206,DIV215109152,2015,2015/10/16 05:00:00+00,1700,A,0,2,1,0,3,0,0,2,3,1,2,1,13,3,2,1,1,1,6,2,2,0,0,3,20,1,RT 259,0,0,0,1,0,0,0,0,0,0,0,0,3.000000,1,0,0,0,0,1,0,8,82,82,MIA,TUD,0,2,2,1,6,NaN,R-VA SR00259SB,9.630000,"586,049.000000",834.000000,38.645544,-78.870556
1,-77.134973,38.943531,2,153060033,20152840041,2015,2015/10/10 05:00:00+00,2350,O,0,0,0,0,0,0,0,1,9,1,5,1,1,1,2,1,1,1,1,6,2,0,0,3,28,4,RT2290/CREST LN,1,0,0,1,0,1,0,1,0,0,0,0,10.000000,1,2,0,0,1,1,1,9,29,29,LOC,TUD,1,5,7,4,8,NOVA,R-VA OT90072NB,0.410000,NaN,NaN,38.943531,-77.134973
2,-79.499399,36.660702,3,153035144,DIV615130489,2015,2015/10/29 05:00:00+00,1124,B,0,0,1,0,1,0,0,1,9,1,2,1,8,3,2,1,1,1,6,1,2,0,0,3,28,1,WHITMELL SCHOOL ROAD,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,1,1,0,1,0,3,71,71,MAC,TUD,0,3,6,1,12,NaN,R-VA071SC00750SB,5.900000,"518,624.000000",137.000000,36.660702,-79.499399
3,-77.336244,38.545770,4,150975078,15053536,2015,2015/04/06 05:00:00+00,900,O,0,0,0,0,0,0,0,2,1,1,2,1,9,1,2,1,2,4,3,1,2,0,0,3,20,1,0619 FULLER ROAD,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,1,0,9,76,76,MAC,TUD,1,3,7,1,8,NOVA,R-VA076SC00619EB,25.740000,NaN,NaN,38.545770,-77.336244
4,-79.337225,36.581073,5,152335159,15007126,2015,2015/08/21 05:00:00+00,751,O,0,0,0,0,0,0,0,2,2,1,2,1,1,1,2,1,2,1,6,1,2,0,0,3,20,1,SOUTH BOSTON RD,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,1,0,3,108,108,OPA,TWD,1,4,6,3,12,DAN,R-VA US00058WB,301.270000,"519,895.000000",876.000000,36.581073,-79.337225


In [15]:
# View dataframe
df_vdot_basic.tail()

,X,Y,OBJECTID,DOCUMENT_NBR,LOCAL_CASE_CD,CRASH_YEAR,CRASH_DT,CRASH_MILITARY_TM,CRASH_SEVERITY,K_PEOPLE,A_PEOPLE,B_PEOPLE,C_PEOPLE,PERSONS_INJURED,PEDESTRIANS_KILLED,PEDESTRIANS_INJURED,VEH_COUNT,COLLISION_TYPE,WEATHER_CONDITION,LIGHT_CONDITION,ROADWAY_SURFACE_COND,RELATION_TO_ROADWAY,ROADWAY_ALIGNMENT,ROADWAY_SURFACE_TYPE,ROADWAY_DEFECT,ROADWAY_DESCRIPTION,INTERSECTION_TYPE,TRAFFIC_CONTROL_TYPE,TRFC_CTRL_STATUS_TYPE,WORK_ZONE_RELATED,WORK_ZONE_LOCATION,WORK_ZONE_TYPE,SCHOOL_ZONE,FIRST_HARMFUL_EVENT,FIRST_HARMFUL_EVENT_LOC,ROUTE_OR_STREET_NM,ALCOHOL_NOTALCOHOL,BELTED_UNBELTED,BIKE_NONBIKE,DISTRACTED_NOTDISTRACTED,ANIMAL,DROWSY_NOTDROWSY,DRUG_NODRUG,GR_NOGR,HITRUN_NOT_HITRUN,LGTRUCK_NONLGTRUCK,MOTOR_NONMOTOR,PED_NONPED,SPEED_DIFF_MAX,SPEED_NOTSPEED,RD_TYPE,INTERSECTION_ANALYSIS,SENIOR_NOTSENIOR,YOUNG_NOTYOUNG,MAINLINE_YN,NIGHT,VDOT_DISTRICT,JURIS_CODE,PHYSICAL_JURIS,FUN,FAC,AREA_TYPE,SYSTEM,VSP,OWNERSHIP,PLAN_DISTRICT,MPO_NAME,RTE_NM,RNS_MP,NODE,OFFSET,LAT,LON
1079554,-79.968982,37.275756,1079555,231805235,23-065148,2023,2023/06/27 05:00:00+00,2332,O,0,0,0,0,0,0,0,2,1,1,5,1,1,3,2,1,1,1,1,6,2,0,0,3,20,1,SALEM AVE SW,0,0,0,0,0,0,0,0,1,0,0,0,20.000000,1,3,0,0,0,1,1,2,128,128,LOC,NaN,1,5,6,3,5,ROAN,S-VA128PR SALEM AVE SW,1.580000,NaN,NaN,37.275756,-79.968982
1079555,-77.231928,38.757442,1079556,230205184,20230170096,2023,2023/01/17 05:00:00+00,1259,O,0,0,0,0,0,0,0,2,2,5,2,2,1,3,2,1,2,1,6,1,2,0,0,3,20,1,FAIRFAX COUNTY PARKWAY,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,3,0,1,0,1,0,9,29,29,OPA,TWD,1,2,7,1,8,NOVA,R-VA SR00286NB,6.480000,NaN,NaN,38.757442,-77.231928
1079556,-77.499265,38.296274,1079557,231675186,2023003319,2023,2023/06/16 05:00:00+00,1104,B,0,0,1,1,2,0,0,2,2,1,2,1,8,1,2,1,2,1,6,1,2,0,0,3,20,1,GATEWAY BLVD,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,1,1,1,1,0,6,111,111,OPA,TWD,1,2,2,1,16,FRED,R-VA SR00003EB,32.360000,"617,401.000000",0.000000,38.296274,-77.499265
1079557,-77.506790,38.291552,1079558,231945258,DIV223058185,2023,2023/07/13 05:00:00+00,1205,O,0,0,0,0,0,0,0,2,1,1,2,1,8,1,2,1,3,1,6,1,2,0,0,3,20,1,NB95,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,1,0,6,111,111,INT,TWD,1,1,2,1,16,FRED,R-VA IS00095NB,130.110000,NaN,NaN,38.291552,-77.506790
1079558,-78.134275,38.935407,1079559,231225139,2304290013,2023,2023/04/29 05:00:00+00,1000,O,0,0,0,0,0,0,0,2,2,1,2,1,9,3,2,1,1,3,4,6,2,0,0,3,20,1,Dismal Hollow Rd,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,1,0,0,1,0,8,93,93,MAC,TUD,0,3,2,1,7,NaN,R-VA093SC00624SB,0.500000,"641,079.000000",11.000000,38.935407,-78.134275


In [16]:
df_vdot_basic.shape, df_vdot_basic.columns, df_vdot_basic.dtypes

((1079559, 73),
 Index(['X', 'Y', 'OBJECTID', 'DOCUMENT_NBR', 'LOCAL_CASE_CD', 'CRASH_YEAR', 'CRASH_DT', 'CRASH_MILITARY_TM', 'CRASH_SEVERITY', 'K_PEOPLE', 'A_PEOPLE', 'B_PEOPLE', 'C_PEOPLE', 'PERSONS_INJURED', 'PEDESTRIANS_KILLED', 'PEDESTRIANS_INJURED', 'VEH_COUNT', 'COLLISION_TYPE', 'WEATHER_CONDITION', 'LIGHT_CONDITION', 'ROADWAY_SURFACE_COND', 'RELATION_TO_ROADWAY', 'ROADWAY_ALIGNMENT', 'ROADWAY_SURFACE_TYPE', 'ROADWAY_DEFECT', 'ROADWAY_DESCRIPTION', 'INTERSECTION_TYPE', 'TRAFFIC_CONTROL_TYPE', 'TRFC_CTRL_STATUS_TYPE', 'WORK_ZONE_RELATED', 'WORK_ZONE_LOCATION', 'WORK_ZONE_TYPE', 'SCHOOL_ZONE', 'FIRST_HARMFUL_EVENT', 'FIRST_HARMFUL_EVENT_LOC', 'ROUTE_OR_STREET_NM', 'ALCOHOL_NOTALCOHOL', 'BELTED_UNBELTED', 'BIKE_NONBIKE', 'DISTRACTED_NOTDISTRACTED', 'ANIMAL', 'DROWSY_NOTDROWSY', 'DRUG_NODRUG', 'GR_NOGR', 'HITRUN_NOT_HITRUN', 'LGTRUCK_NONLGTRUCK', 'MOTOR_NONMOTOR', 'PED_NONPED', 'SPEED_DIFF_MAX', 'SPEED_NOTSPEED', 'RD_TYPE', 'INTERSECTION_ANALYSIS', 'SENIOR_NOTSENIOR', 'YOUNG_NOTYOUN

In [137]:
# Show a summary of the number of missing values (NaN or null) in each column of the df_vdot_basic DataFrame.
df_vdot_basic.isna().sum()

X                               15
Y                               15
OBJECTID                         0
DOCUMENT_NBR                     0
LOCAL_CASE_CD                 3490
CRASH_YEAR                       0
CRASH_DT                         0
CRASH_MILITARY_TM                0
CRASH_SEVERITY                   0
K_PEOPLE                         0
A_PEOPLE                         0
B_PEOPLE                         0
C_PEOPLE                         0
PERSONS_INJURED                  0
PEDESTRIANS_KILLED               0
PEDESTRIANS_INJURED              0
VEH_COUNT                        0
COLLISION_TYPE                   0
WEATHER_CONDITION                0
LIGHT_CONDITION                  0
ROADWAY_SURFACE_COND             0
RELATION_TO_ROADWAY              0
ROADWAY_ALIGNMENT                0
ROADWAY_SURFACE_TYPE             0
ROADWAY_DEFECT                   0
ROADWAY_DESCRIPTION              0
INTERSECTION_TYPE                0
TRAFFIC_CONTROL_TYPE             0
TRFC_CTRL_STATUS_TYP

## Merge df_vdot_basic and df_vdot_detailed

In [110]:
# Check the common colummns between 2 dataframes for merging
common_columns = df_vdot_basic.columns.intersection(df_vdot_detailed.columns).tolist()

# Print the common columns
print("Common Columns:", common_columns)

Common Columns: ['OBJECTID', 'DOCUMENT_NBR', 'LOCAL_CASE_CD', 'CRASH_YEAR', 'CRASH_SEVERITY', 'ROUTE_OR_STREET_NM']


In [111]:
# Merge dataframes
df_vdot = df_vdot_basic.merge(df_vdot_detailed.drop(columns='OBJECTID'), how="inner", on=['DOCUMENT_NBR', 'LOCAL_CASE_CD', 'CRASH_YEAR', 'CRASH_SEVERITY', 'ROUTE_OR_STREET_NM'])

# View dataframe
df_vdot.head()

,X,Y,OBJECTID,DOCUMENT_NBR,LOCAL_CASE_CD,CRASH_YEAR,CRASH_DT,CRASH_MILITARY_TM,CRASH_SEVERITY,K_PEOPLE,A_PEOPLE,B_PEOPLE,C_PEOPLE,PERSONS_INJURED,PEDESTRIANS_KILLED,PEDESTRIANS_INJURED,VEH_COUNT,COLLISION_TYPE,WEATHER_CONDITION,LIGHT_CONDITION,ROADWAY_SURFACE_COND,RELATION_TO_ROADWAY,ROADWAY_ALIGNMENT,ROADWAY_SURFACE_TYPE,ROADWAY_DEFECT,ROADWAY_DESCRIPTION,INTERSECTION_TYPE,TRAFFIC_CONTROL_TYPE,TRFC_CTRL_STATUS_TYPE,WORK_ZONE_RELATED,WORK_ZONE_LOCATION,WORK_ZONE_TYPE,SCHOOL_ZONE,FIRST_HARMFUL_EVENT,FIRST_HARMFUL_EVENT_LOC,ROUTE_OR_STREET_NM,ALCOHOL_NOTALCOHOL,BELTED_UNBELTED,BIKE_NONBIKE,DISTRACTED_NOTDISTRACTED,ANIMAL,DROWSY_NOTDROWSY,DRUG_NODRUG,GR_NOGR,HITRUN_NOT_HITRUN,LGTRUCK_NONLGTRUCK,MOTOR_NONMOTOR,PED_NONPED,SPEED_DIFF_MAX,SPEED_NOTSPEED,RD_TYPE,INTERSECTION_ANALYSIS,SENIOR_NOTSENIOR,YOUNG_NOTYOUNG,MAINLINE_YN,NIGHT,VDOT_DISTRICT,JURIS_CODE,PHYSICAL_JURIS,FUN,FAC,AREA_TYPE,SYSTEM,VSP,OWNERSHIP,PLAN_DISTRICT,MPO_NAME,RTE_NM,RNS_MP,NODE,OFFSET,LAT,LON,DRIVER_VEHICLENUMBER,DRIVER_INJURYTYPE,DRIVER_AGE,DRIVER_GENDER,DRIVER_ACTION_TYPE_CD,DRIVER_AIRBAG_DEPLOYMENT,DRIVER_ALCOHOL_TEST_TYPE_CD,DRIVER_CONDITION_TYPE_CD,DRIVER_DISTRACTION_TYPE_CD,DRIVER_DRINKING_TYPE_CD,DRIVER_DRUG_USE_CD,DRIVER_EJECTED_FROM_VEHICLE,DRIVER_EMS_TRANSPORT_IND,DRIVER_FLED_SCENE_IND,DRIVER_SAFETY_EQUIP_USED,DRIVER_VIS_OBSCURED_TYPE_CD,SUMMONS_ISSUED_CD,VEHICLENUMBER,VEHICLE_BODY_TYPE_CD,VEHICLE_MAKE_NM,VEHICLE_MANEUVER_TYPE_CD,VEHICLE_MODEL_NM,VEHICLE_YEAR_NBR,COMM_CARGO_BODY_TYPE_CD,COMM_VEHICLE_BODY_TYPE_CD,SPEED_BEFORE,SPEED_POSTED,SPEED_MAX_SAFE,FIRST_CRASH_EVENT_CD,SECOND_CRASH_EVENT_CD,THIRD_CRASH_EVENT_CD,FOURTH_CRASH_EVENT_CD,MOST_HARMFUL_CRASH_EVENT_CD,INITIAL_VEH_IMPACT_AREA_CD,DIRECTION_OF_TRAVEL_CD,PASS_VEHICLENUMBER,PASS_INJURYTYPE,PASS_AGE,PASS_GENDER,PASS_SAFETY_EQUIP_USED,PASS_AIRBAG_DEPLOYMENT,PASS_EJECTED_FROM_VEHICLE,PASS_EMS_TRANSPORT_IND,BIKE_VEHICLENUMBER,BIKE_INJURYTYPE,BIKE_AGE,BIKE_GENDER,PED_NUMBER,PED_INJURYTYPE,PED_AGE,PED_GENDER,PED_ACTION,PED_DRINK,PED_COND,PED_AL_TEST,PED_DRUG,PED_RFLCT,NODE_TOTAADT,SEG_TOTAADT,BASETYPEDESC,CURBGUTTERDESC,RURALURBANDESC,ACCESS_CONTROL,SURFACEDESC,GOVCONDESC,SIDEWALKDESC,TRUCKCOMMR,NUMBEROFLANE,CARSPEEDLIMIT,PAVEMENT_CONDITIONVALUE,PAVEMENT_ROUGHNESSVALUE,PAVEMENT_WIDTH,RIGHTSHOULDERWIDTH,LEFTSHOULDERWIDTH,MEDIAN_TYPE,MEDIAN_TYPEDESC,MEDIAN_LEFTSHOULDERWIDTH,MEDIAN_RIGHTSHOULDERWIDTH,MEDIAN_WIDTHMAX,MEDIAN_WIDTHMIN
0,-78.870556,38.645544,1,152915206,DIV215109152,2015,2015/10/16 05:00:00+00,1700,A,0,2,1,0,3,0,0,2,3,1,2,1,13,3,2,1,1,1,6,2,2,0,0,3,20,1,RT 259,0,0,0,1,0,0,0,0,0,0,0,0,3.000000,1,0,0,0,0,1,0,8,82,82,MIA,TUD,0,2,2,1,6,NaN,R-VA SR00259SB,9.630000,"586,049.000000",834.000000,38.645544,-78.870556,1;2,A;PDO,26;56,Male;Male,10. Wrong Side Of Road - Not Overtaking;2. Exc...,1. Deployed - Front;1. Deployed - Front,n/a; n/a,1. No Defects;1. No Defects,7. Eyes Not on Road;14. No Distraction,1. No Drinking;1. No Drinking,2. No;2. No,1. Not Ejected;1. Not Ejected,Yes;No,No;No,3. Lap and Shoulder Belt;3. Lap and Shoulder Belt,1. Not Obscured;1. Not Obscured,1. Yes;2. No,1;2,1. Passenger car;22. Truck (SUV),HONDA;FORD,3. Making Left Turn;1. Going Straight Ahead,CIVIC;EXPLORER,2000;2000,n/a; n/a,n/a; n/a,25;58,55;55,0;55,20. Motor Vehicle In Transport;20. Motor Vehic...,n/a; n/a,n/a; n/a,n/a; n/a,20. Motor Vehicle In Transport;20. Motor Vehic...,12;12,East;West,1;2,A;B,28;11,Female;Female,3. Lap and Shoulder Belt;3. Lap and Shoulder Belt,1. Deployed - Front;1. Deployed - Front,1. Not Ejected;1. Not Ejected,Yes;No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"5,859.000000","5,403.000000",1.000000,1.000000,1.000000,1.000000,NaN,1,1.000000,1.000000,2.000000,55.000000,0.000000,84.000000,20.000000,2.000000,2.000000,2.000000,NaN,0.000000,0.000000,0.000000,0.000000
1,-77.134973,38.943531,2,153060033,20152840041,2015,2015/10/10 05:00:00+00,2350,O,0,0,0,0,0,0,0,1,9,1,5,1,1,1,2,1,1,1,1,6,2,0,0,3,28,4,RT2290/CREST LN,1,0,0,1,0,1,0,1,0,0,0,0,10.000000,1,2,0,0,1,1,1,9,29,29,LOC,TUD,1,5,7,4,8

In [112]:
# Convert 'CRASH_YEAR' to a numeric column
df_vdot['CRASH_YEAR'] = df_vdot['CRASH_YEAR'].astype(int)

# Keep only rows from 2017 to 2021
df_vdot_filtered = df_vdot[(df_vdot['CRASH_YEAR'] >= 2017) & (df_vdot['CRASH_YEAR'] <= 2021)]

# Reset index
df_vdot_filtered.reset_index(drop=True, inplace=True)

# View the filtered dataframe
df_vdot_filtered.head()

,X,Y,OBJECTID,DOCUMENT_NBR,LOCAL_CASE_CD,CRASH_YEAR,CRASH_DT,CRASH_MILITARY_TM,CRASH_SEVERITY,K_PEOPLE,A_PEOPLE,B_PEOPLE,C_PEOPLE,PERSONS_INJURED,PEDESTRIANS_KILLED,PEDESTRIANS_INJURED,VEH_COUNT,COLLISION_TYPE,WEATHER_CONDITION,LIGHT_CONDITION,ROADWAY_SURFACE_COND,RELATION_TO_ROADWAY,ROADWAY_ALIGNMENT,ROADWAY_SURFACE_TYPE,ROADWAY_DEFECT,ROADWAY_DESCRIPTION,INTERSECTION_TYPE,TRAFFIC_CONTROL_TYPE,TRFC_CTRL_STATUS_TYPE,WORK_ZONE_RELATED,WORK_ZONE_LOCATION,WORK_ZONE_TYPE,SCHOOL_ZONE,FIRST_HARMFUL_EVENT,FIRST_HARMFUL_EVENT_LOC,ROUTE_OR_STREET_NM,ALCOHOL_NOTALCOHOL,BELTED_UNBELTED,BIKE_NONBIKE,DISTRACTED_NOTDISTRACTED,ANIMAL,DROWSY_NOTDROWSY,DRUG_NODRUG,GR_NOGR,HITRUN_NOT_HITRUN,LGTRUCK_NONLGTRUCK,MOTOR_NONMOTOR,PED_NONPED,SPEED_DIFF_MAX,SPEED_NOTSPEED,RD_TYPE,INTERSECTION_ANALYSIS,SENIOR_NOTSENIOR,YOUNG_NOTYOUNG,MAINLINE_YN,NIGHT,VDOT_DISTRICT,JURIS_CODE,PHYSICAL_JURIS,FUN,FAC,AREA_TYPE,SYSTEM,VSP,OWNERSHIP,PLAN_DISTRICT,MPO_NAME,RTE_NM,RNS_MP,NODE,OFFSET,LAT,LON,DRIVER_VEHICLENUMBER,DRIVER_INJURYTYPE,DRIVER_AGE,DRIVER_GENDER,DRIVER_ACTION_TYPE_CD,DRIVER_AIRBAG_DEPLOYMENT,DRIVER_ALCOHOL_TEST_TYPE_CD,DRIVER_CONDITION_TYPE_CD,DRIVER_DISTRACTION_TYPE_CD,DRIVER_DRINKING_TYPE_CD,DRIVER_DRUG_USE_CD,DRIVER_EJECTED_FROM_VEHICLE,DRIVER_EMS_TRANSPORT_IND,DRIVER_FLED_SCENE_IND,DRIVER_SAFETY_EQUIP_USED,DRIVER_VIS_OBSCURED_TYPE_CD,SUMMONS_ISSUED_CD,VEHICLENUMBER,VEHICLE_BODY_TYPE_CD,VEHICLE_MAKE_NM,VEHICLE_MANEUVER_TYPE_CD,VEHICLE_MODEL_NM,VEHICLE_YEAR_NBR,COMM_CARGO_BODY_TYPE_CD,COMM_VEHICLE_BODY_TYPE_CD,SPEED_BEFORE,SPEED_POSTED,SPEED_MAX_SAFE,FIRST_CRASH_EVENT_CD,SECOND_CRASH_EVENT_CD,THIRD_CRASH_EVENT_CD,FOURTH_CRASH_EVENT_CD,MOST_HARMFUL_CRASH_EVENT_CD,INITIAL_VEH_IMPACT_AREA_CD,DIRECTION_OF_TRAVEL_CD,PASS_VEHICLENUMBER,PASS_INJURYTYPE,PASS_AGE,PASS_GENDER,PASS_SAFETY_EQUIP_USED,PASS_AIRBAG_DEPLOYMENT,PASS_EJECTED_FROM_VEHICLE,PASS_EMS_TRANSPORT_IND,BIKE_VEHICLENUMBER,BIKE_INJURYTYPE,BIKE_AGE,BIKE_GENDER,PED_NUMBER,PED_INJURYTYPE,PED_AGE,PED_GENDER,PED_ACTION,PED_DRINK,PED_COND,PED_AL_TEST,PED_DRUG,PED_RFLCT,NODE_TOTAADT,SEG_TOTAADT,BASETYPEDESC,CURBGUTTERDESC,RURALURBANDESC,ACCESS_CONTROL,SURFACEDESC,GOVCONDESC,SIDEWALKDESC,TRUCKCOMMR,NUMBEROFLANE,CARSPEEDLIMIT,PAVEMENT_CONDITIONVALUE,PAVEMENT_ROUGHNESSVALUE,PAVEMENT_WIDTH,RIGHTSHOULDERWIDTH,LEFTSHOULDERWIDTH,MEDIAN_TYPE,MEDIAN_TYPEDESC,MEDIAN_LEFTSHOULDERWIDTH,MEDIAN_RIGHTSHOULDERWIDTH,MEDIAN_WIDTHMAX,MEDIAN_WIDTHMIN
0,-77.385995,37.542599,254326,170725320,170313135,2017,2017/03/13 05:00:00+00,1545,C,0,0,0,1,1,0,0,2,4,1,2,1,10,5,2,1,2,3,6,1,2,0,0,3,20,1,NINE MILE RD,0,0,0,1,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,1,0,4,43,43,MIA,TUD,1,2,1,1,15,RICH,R-VA SR00033WB,4.030000,"378,250.000000","1,251.000000",37.542599,-77.385995,1;2,PDO;C,33;41,Female;Male,42. Improper or Unsafe Lane Change;1. No Impro...,2. Not Deployed;2. Not Deployed,n/a; n/a,1. No Defects;1. No Defects,12. Navigation Device;14. No Distraction,1. No Drinking;1. No Drinking,2. No;2. No,1. Not Ejected;1. Not Ejected,No;No,No;No,3. Lap and Shoulder Belt;3. Lap and Shoulder Belt,1. Not Obscured;1. Not Obscured,1. Yes;2. No,1;2,1. Passenger car;22. Truck (SUV),MINI;FORD,14. Changing Lanes;1. Going Straight Ahead,COOPER;FLEX,2011;2009,n/a; n/a,n/a; n/a,40;40,40;40,40;40,20. Motor Vehicle In Transport;20. Motor Vehic...,n/a; n/a,n/a; n/a,n/a; n/a,20. Motor Vehicle In Transport;20. Motor Vehic...,1;10,West;West,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"26,091.000000","23,570.000000",1.000000,5.000000,4.000000,1.000000,NaN,1.000000,3.000000,1.000000,4.000000,40.000000,0.000000,150.000000,65.000000,0.000000,0.000000,2.000000,NaN,0.000000,0.000000,0.000000,0.000000
1,-76.823141,36.782158,254327,173135102,2017-02334,2017,2017/11/02 05:00:00+00,628,O,0,0,0,0,0,0,0,1,10,1,5,1,1,1,2,1,2,1,6,1,2,0,0,3,23,1,WALTERS HIGHWAY,0,0,0,0,1,0,0,0,0,0,0,0,NaN,0,0,1,0,0,1,1,5,46,46,MIA,TUD,0,2,5,1,23,HAMP,R-VA US00258EB,25.450000,"393,477.000000",84.000000,36.782158,-76.823141,1,PDO,28,Female,1. No Improper Actio

## Merge df_vdot_filtered and  VDOT’s Virginia Jurisdiction Boundary Data

In [113]:
# Load dataframe
vdot_juris = pd.read_csv('./data/VDOT_JURISDICTION.csv')

In [114]:
# Rename column
vdot_juris.rename(columns={'CODE': 'JURIS_CODE'}, inplace=True)

In [124]:
# Mapping function for general replacements
def replace_jurisdiction(value):
    if 'City of' in value:
        return value.replace('City of', '') + ' City'
    elif 'Town of' in value:
        return value.replace('Town of', '') + ' Town'
    else:
        return value

# Apply the mapping function to the 'JURISDICTION' column
vdot_juris['JURISDICTION'] = vdot_juris['JURISDICTION'].apply(replace_jurisdiction)

# Lower JURISDICTION column
vdot_juris['JURISDICTION'] = vdot_juris['JURISDICTION'].str.lower()

# Display DataFrame
vdot_juris.head()

,JURIS_CODE,JURISDICTION
0,0,arlington county
1,1,accomack county
2,2,albemarle county
3,3,alleghany county
4,4,amelia county


In [120]:
# Merge dataframes
vdot_crash_df = df_vdot_filtered.merge(vdot_juris, how="left", on='JURIS_CODE')

# View dataframe
vdot_crash_df.head()

,X,Y,OBJECTID,DOCUMENT_NBR,LOCAL_CASE_CD,CRASH_YEAR,CRASH_DT,CRASH_MILITARY_TM,CRASH_SEVERITY,K_PEOPLE,A_PEOPLE,B_PEOPLE,C_PEOPLE,PERSONS_INJURED,PEDESTRIANS_KILLED,PEDESTRIANS_INJURED,VEH_COUNT,COLLISION_TYPE,WEATHER_CONDITION,LIGHT_CONDITION,ROADWAY_SURFACE_COND,RELATION_TO_ROADWAY,ROADWAY_ALIGNMENT,ROADWAY_SURFACE_TYPE,ROADWAY_DEFECT,ROADWAY_DESCRIPTION,INTERSECTION_TYPE,TRAFFIC_CONTROL_TYPE,TRFC_CTRL_STATUS_TYPE,WORK_ZONE_RELATED,WORK_ZONE_LOCATION,WORK_ZONE_TYPE,SCHOOL_ZONE,FIRST_HARMFUL_EVENT,FIRST_HARMFUL_EVENT_LOC,ROUTE_OR_STREET_NM,ALCOHOL_NOTALCOHOL,BELTED_UNBELTED,BIKE_NONBIKE,DISTRACTED_NOTDISTRACTED,ANIMAL,DROWSY_NOTDROWSY,DRUG_NODRUG,GR_NOGR,HITRUN_NOT_HITRUN,LGTRUCK_NONLGTRUCK,MOTOR_NONMOTOR,PED_NONPED,SPEED_DIFF_MAX,SPEED_NOTSPEED,RD_TYPE,INTERSECTION_ANALYSIS,SENIOR_NOTSENIOR,YOUNG_NOTYOUNG,MAINLINE_YN,NIGHT,VDOT_DISTRICT,JURIS_CODE,PHYSICAL_JURIS,FUN,FAC,AREA_TYPE,SYSTEM,VSP,OWNERSHIP,PLAN_DISTRICT,MPO_NAME,RTE_NM,RNS_MP,NODE,OFFSET,LAT,LON,DRIVER_VEHICLENUMBER,DRIVER_INJURYTYPE,DRIVER_AGE,DRIVER_GENDER,DRIVER_ACTION_TYPE_CD,DRIVER_AIRBAG_DEPLOYMENT,DRIVER_ALCOHOL_TEST_TYPE_CD,DRIVER_CONDITION_TYPE_CD,DRIVER_DISTRACTION_TYPE_CD,DRIVER_DRINKING_TYPE_CD,DRIVER_DRUG_USE_CD,DRIVER_EJECTED_FROM_VEHICLE,DRIVER_EMS_TRANSPORT_IND,DRIVER_FLED_SCENE_IND,DRIVER_SAFETY_EQUIP_USED,DRIVER_VIS_OBSCURED_TYPE_CD,SUMMONS_ISSUED_CD,VEHICLENUMBER,VEHICLE_BODY_TYPE_CD,VEHICLE_MAKE_NM,VEHICLE_MANEUVER_TYPE_CD,VEHICLE_MODEL_NM,VEHICLE_YEAR_NBR,COMM_CARGO_BODY_TYPE_CD,COMM_VEHICLE_BODY_TYPE_CD,SPEED_BEFORE,SPEED_POSTED,SPEED_MAX_SAFE,FIRST_CRASH_EVENT_CD,SECOND_CRASH_EVENT_CD,THIRD_CRASH_EVENT_CD,FOURTH_CRASH_EVENT_CD,MOST_HARMFUL_CRASH_EVENT_CD,INITIAL_VEH_IMPACT_AREA_CD,DIRECTION_OF_TRAVEL_CD,PASS_VEHICLENUMBER,PASS_INJURYTYPE,PASS_AGE,PASS_GENDER,PASS_SAFETY_EQUIP_USED,PASS_AIRBAG_DEPLOYMENT,PASS_EJECTED_FROM_VEHICLE,PASS_EMS_TRANSPORT_IND,BIKE_VEHICLENUMBER,BIKE_INJURYTYPE,BIKE_AGE,BIKE_GENDER,PED_NUMBER,PED_INJURYTYPE,PED_AGE,PED_GENDER,PED_ACTION,PED_DRINK,PED_COND,PED_AL_TEST,PED_DRUG,PED_RFLCT,NODE_TOTAADT,SEG_TOTAADT,BASETYPEDESC,CURBGUTTERDESC,RURALURBANDESC,ACCESS_CONTROL,SURFACEDESC,GOVCONDESC,SIDEWALKDESC,TRUCKCOMMR,NUMBEROFLANE,CARSPEEDLIMIT,PAVEMENT_CONDITIONVALUE,PAVEMENT_ROUGHNESSVALUE,PAVEMENT_WIDTH,RIGHTSHOULDERWIDTH,LEFTSHOULDERWIDTH,MEDIAN_TYPE,MEDIAN_TYPEDESC,MEDIAN_LEFTSHOULDERWIDTH,MEDIAN_RIGHTSHOULDERWIDTH,MEDIAN_WIDTHMAX,MEDIAN_WIDTHMIN,JURISDICTION
0,-77.385995,37.542599,254326,170725320,170313135,2017,2017/03/13 05:00:00+00,1545,C,0,0,0,1,1,0,0,2,4,1,2,1,10,5,2,1,2,3,6,1,2,0,0,3,20,1,NINE MILE RD,0,0,0,1,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,1,0,4,43,43,MIA,TUD,1,2,1,1,15,RICH,R-VA SR00033WB,4.030000,"378,250.000000","1,251.000000",37.542599,-77.385995,1;2,PDO;C,33;41,Female;Male,42. Improper or Unsafe Lane Change;1. No Impro...,2. Not Deployed;2. Not Deployed,n/a; n/a,1. No Defects;1. No Defects,12. Navigation Device;14. No Distraction,1. No Drinking;1. No Drinking,2. No;2. No,1. Not Ejected;1. Not Ejected,No;No,No;No,3. Lap and Shoulder Belt;3. Lap and Shoulder Belt,1. Not Obscured;1. Not Obscured,1. Yes;2. No,1;2,1. Passenger car;22. Truck (SUV),MINI;FORD,14. Changing Lanes;1. Going Straight Ahead,COOPER;FLEX,2011;2009,n/a; n/a,n/a; n/a,40;40,40;40,40;40,20. Motor Vehicle In Transport;20. Motor Vehic...,n/a; n/a,n/a; n/a,n/a; n/a,20. Motor Vehicle In Transport;20. Motor Vehic...,1;10,West;West,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"26,091.000000","23,570.000000",1.000000,5.000000,4.000000,1.000000,NaN,1.000000,3.000000,1.000000,4.000000,40.000000,0.000000,150.000000,65.000000,0.000000,0.000000,2.000000,NaN,0.000000,0.000000,0.000000,0.000000,henrico county
1,-76.823141,36.782158,254327,173135102,2017-02334,2017,2017/11/02 05:00:00+00,628,O,0,0,0,0,0,0,0,1,10,1,5,1,1,1,2,1,2,1,6,1,2,0,0,3,23,1,WALTERS HIGHWAY,0,0,0,0,1,0,0,0,0,0,0,0,NaN,0,0,1,0,0,1,1,5,46,46,MIA,TUD,0,2,5,1,23,HAMP,R-VA US00258EB,25.450000,"393,477.000000",84.000000,36.782158,-76.823141,1,PDO,28

## Merge vdot_crash_df and ACS Data

In [125]:
# Lower COUNTY_NAME column
virginia_acs['COUNTY_NAME'] = virginia_acs['COUNTY_NAME'].str.lower()

In [126]:
# Merge dataframes
vdot_crash = virginia_acs.merge(vdot_crash_df, how="inner", left_on='COUNTY_NAME', right_on='JURISDICTION')

In [132]:
# Reset index
vdot_crash.reset_index(inplace=True, drop=True)

# View dataframe
vdot_crash

,NAME,POPULATION,BIKE_TO_WORK_EST,BIKE_TO_WORK_MARG,WALK_TO_WORK_EST,WALK_TO_WORK_MARG,DRIVE_SOLO_TO_WORK_EST,DRIVE_SOLO_TO_WORK_MARG,CARPOOL_TO_WORK_EST,CARPOOL_TO_WORK_MARG,PUBTRANS_TO_WORK_EST,PUBTRANS_TO_WORK_MARG,state,county,county_subdivision,MUNI_NAME,COUNTY_NAME,X,Y,OBJECTID,DOCUMENT_NBR,LOCAL_CASE_CD,CRASH_YEAR,CRASH_DT,CRASH_MILITARY_TM,CRASH_SEVERITY,K_PEOPLE,A_PEOPLE,B_PEOPLE,C_PEOPLE,PERSONS_INJURED,PEDESTRIANS_KILLED,PEDESTRIANS_INJURED,VEH_COUNT,COLLISION_TYPE,WEATHER_CONDITION,LIGHT_CONDITION,ROADWAY_SURFACE_COND,RELATION_TO_ROADWAY,ROADWAY_ALIGNMENT,ROADWAY_SURFACE_TYPE,ROADWAY_DEFECT,ROADWAY_DESCRIPTION,INTERSECTION_TYPE,TRAFFIC_CONTROL_TYPE,TRFC_CTRL_STATUS_TYPE,WORK_ZONE_RELATED,WORK_ZONE_LOCATION,WORK_ZONE_TYPE,SCHOOL_ZONE,FIRST_HARMFUL_EVENT,FIRST_HARMFUL_EVENT_LOC,ROUTE_OR_STREET_NM,ALCOHOL_NOTALCOHOL,BELTED_UNBELTED,BIKE_NONBIKE,DISTRACTED_NOTDISTRACTED,ANIMAL,DROWSY_NOTDROWSY,DRUG_NODRUG,GR_NOGR,HITRUN_NOT_HITRUN,LGTRUCK_NONLGTRUCK,MOTOR_NONMOTOR,PED_NONPED,SPEED_DIFF_MAX,SPEED_NOTSPEED,RD_TYPE,INTERSECTION_ANALYSIS,SENIOR_NOTSENIOR,YOUNG_NOTYOUNG,MAINLINE_YN,NIGHT,VDOT_DISTRICT,JURIS_CODE,PHYSICAL_JURIS,FUN,FAC,AREA_TYPE,SYSTEM,VSP,OWNERSHIP,PLAN_DISTRICT,MPO_NAME,RTE_NM,RNS_MP,NODE,OFFSET,LAT,LON,DRIVER_VEHICLENUMBER,DRIVER_INJURYTYPE,DRIVER_AGE,DRIVER_GENDER,DRIVER_ACTION_TYPE_CD,DRIVER_AIRBAG_DEPLOYMENT,DRIVER_ALCOHOL_TEST_TYPE_CD,DRIVER_CONDITION_TYPE_CD,DRIVER_DISTRACTION_TYPE_CD,DRIVER_DRINKING_TYPE_CD,DRIVER_DRUG_USE_CD,DRIVER_EJECTED_FROM_VEHICLE,DRIVER_EMS_TRANSPORT_IND,DRIVER_FLED_SCENE_IND,DRIVER_SAFETY_EQUIP_USED,DRIVER_VIS_OBSCURED_TYPE_CD,SUMMONS_ISSUED_CD,VEHICLENUMBER,VEHICLE_BODY_TYPE_CD,VEHICLE_MAKE_NM,VEHICLE_MANEUVER_TYPE_CD,VEHICLE_MODEL_NM,VEHICLE_YEAR_NBR,COMM_CARGO_BODY_TYPE_CD,COMM_VEHICLE_BODY_TYPE_CD,SPEED_BEFORE,SPEED_POSTED,SPEED_MAX_SAFE,FIRST_CRASH_EVENT_CD,SECOND_CRASH_EVENT_CD,THIRD_CRASH_EVENT_CD,FOURTH_CRASH_EVENT_CD,MOST_HARMFUL_CRASH_EVENT_CD,INITIAL_VEH_IMPACT_AREA_CD,DIRECTION_OF_TRAVEL_CD,PASS_VEHICLENUMBER,PASS_INJURYTYPE,PASS_AGE,PASS_GENDER,PASS_SAFETY_EQUIP_USED,PASS_AIRBAG_DEPLOYMENT,PASS_EJECTED_FROM_VEHICLE,PASS_EMS_TRANSPORT_IND,BIKE_VEHICLENUMBER,BIKE_INJURYTYPE,BIKE_AGE,BIKE_GENDER,PED_NUMBER,PED_INJURYTYPE,PED_AGE,PED_GENDER,PED_ACTION,PED_DRINK,PED_COND,PED_AL_TEST,PED_DRUG,PED_RFLCT,NODE_TOTAADT,SEG_TOTAADT,BASETYPEDESC,CURBGUTTERDESC,RURALURBANDESC,ACCESS_CONTROL,SURFACEDESC,GOVCONDESC,SIDEWALKDESC,TRUCKCOMMR,NUMBEROFLANE,CARSPEEDLIMIT,PAVEMENT_CONDITIONVALUE,PAVEMENT_ROUGHNESSVALUE,PAVEMENT_WIDTH,RIGHTSHOULDERWIDTH,LEFTSHOULDERWIDTH,MEDIAN_TYPE,MEDIAN_TYPEDESC,MEDIAN_LEFTSHOULDERWIDTH,MEDIAN_RIGHTSHOULDERWIDTH,MEDIAN_WIDTHMAX,MEDIAN_WIDTHMIN,JURISDICTION
0,"Alexandria city, Alexandria city",158185,1013,189,3390,437,53121,1436,7001,880,15729,857,51,510,90020,Alexandria city,alexandria city,-77.075017,38.800837,254465,172775032,DIV717133123,2017,2017/09/30 05:00:00+00,403,O,0,0,0,0,0,0,0,1,9,1,4,1,5,2,2,1,4,1,6,1,2,0,0,3,3,5,I-495 WEST,0,0,0,1,0,0,0,0,1,0,0,0,20.000000,1,1,0,0,0,0,1,9,100,100,INT,OUD,1,1,7,1,8,NOVA,R-VA IS00095SB RMP175.00B,0.100000,NaN,NaN,38.800837,-77.075017,1,PDO,NaN,n/a,40. Fail to Maintain Proper Control,n/a,n/a,9. Unknown,11. Other,6. Unknown,3. Unknown,n/a,NotProvided,Yes,n/a,13. Other,n/a,1,1. Passenger car,BMW,10. Ran Off Road - Left,7-SERIES,2010,n/a,n/a,55,55,35,28. Ran Off Road,"8. Sign, Traffic Signal",3. Utility Pole,n/a,3. Utility Pole,12,n/a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000,4.000000,1.000000,NaN,1,1.000000,1.000000,1.000000,NaN,0.000000,0.000000,0.000000,6.000000,3.000000,1.000000,NaN,0.000000,0.000000,0.000000,0.000000,alexandria city
1,"Alexandria city, Alexandria city",158185,1013,189,3390,437,53121,1436,7001,880,15729,857,51,510,90020,Alexandria city,alexandria city,-77.116559,38.833541,254492,173485274,17-214911,2017,2017/12/14 05:00:00+00,1600,O,0,0,0,0,0,0,0,2,1,1,2,1,6,2,2,1,1,2,8,1,2,0,0,3,20,1,SEMINARY RD,0,0,0,1,0,0,0,0,0,0,

- Sum up the value by group_by
- Variables to use:
+ Crash Severity | CRASH_SEVERITY | CRASHDATA_DETAILS
+ Pedestrian-Related Crash | PED_NONPED | CRASHDATA_BASIC
+ Bicycle-Related Crash | BIKE_NONBIKE | CRASHDATA_BASIC
+ Heavy Vehicle Crash | LGTRUCK_NONLGTRUCK | CRASHDATA_BASIC